# 1 Importing packages

In [1]:
import pandas as pd
from sqlalchemy import create_engine

import sqlite3
import datetime

# Function converts sql query to dataframe

In [10]:
def sql_to_df(database, query):
    conn=sqlite3.connect(database) # connects to database
    cursor = conn.cursor()
    
    cursor.execute(query) # executes query
    
    results = cursor.fetchall() # retrieves all of the rows of the query
    
    df=pd.DataFrame(results) # converts the query into a data frame
    
    # collects the column names from the queries and assigns those names to the column of the dataframe
    colname=[]
    
    for col in cursor.description: # creates a list for the column names
        colname.append(col[0])
        
    colname_dict={}
    
    for ent in range(len(cursor.description)): # creates a dictionary with the column numbers as the index
        colname_dict[ent]=colname[ent]
        
    df.rename(colname_dict, axis=1, inplace=True) # renames the dataframe columns with the columns for the sql table
    
    conn.close() # closes the connection
    
    return df   
    

# Question 10

In [15]:
query = "select name, sum(r.cost) as revenue \
    from (select name, case when m.memid =0 then f.guestcost \
                else f.membercost \
            end cost \
    from Bookings as b \
                   left join Facilities as f \
        on b.facid = f.facid \
        left join Members as m \
        on b.memid = m.memid) as r \
    group by name \
    having revenue <1000 \
    order by revenue;"

database='sqlite_db_pythonsqlite.db'

q10=sql_to_df(database, query)

q10.head()

,name,revenue
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5


# Question 11

In [29]:
query="select m1.memid, m1.surname, m1.firstname, m1.recommendedby, m2.surname, m2.firstname \
        from Members as m1 \
            left join Members as m2 \
            on m1.recommendedby = m2.memid \
        order by m1.surname, m1.firstname;"

q11=sql_to_df(database, query)

q11

,memid,surname,firstname,recommendedby,surname,firstname
0,15,Bader,Florence,9,Stibbons,Ponder
1,12,Baker,Anne,9,Stibbons,Ponder
2,16,Baker,Timothy,13,Farrell,Jemima
3,8,Boothe,Tim,3,Rownam,Tim
4,5,Butters,Gerald,1,Smith,Darren
5,22,Coplin,Joan,16,Baker,Timothy
6,36,Crumpet,Erica,2,Smith,Tracy
7,7,Dare,Nancy,4,Joplette,Janice
8,28,Farrell,David,,None,None
9,13,Farrell,Jemima,,None,None


# Question 12

In [56]:
query="select name as facility_name, count(b.facid) as number_of_uses \
        from Bookings as b \
            left join Facilities as f \
            on b.facid = f.facid \
        where b.memid <> 0 \
        group by b.facid;"

sql_to_df(database, query)

,facility_name,number_of_uses
0,Tennis Court 1,308
1,Tennis Court 2,276
2,Badminton Court,344
3,Table Tennis,385
4,Massage Room 1,421
5,Massage Room 2,27
6,Squash Court,195
7,Snooker Table,421
8,Pool Table,783


# Question 13

In [74]:
query="select b.facid as facid, name, strftime('%m', starttime) as month, count(b.memid) as number_of_uses \
            from Bookings as b \
                left join Facilities as f \
                    on b.facid = f.facid \
                left join Members as m \
                    on b.memid = m.memid \
            where b.memid <> 0 \
            group by b.facid, month;"

sql_to_df(database, query)

,facid,name,month,number_of_uses
0,0,Tennis Court 1,07,65
1,0,Tennis Court 1,08,111
2,0,Tennis Court 1,09,132
3,1,Tennis Court 2,07,41
4,1,Tennis Court 2,08,109
5,1,Tennis Court 2,09,126
6,2,Badminton Court,07,51
7,2,Badminton Court,08,132
8,2,Badminton Court,09,161
9,3,Table Tennis,07,48
